RNN(Recurrent Neural Network)也叫**循环神经网络/递归神经网络**

运用于语音识别，自然语言处理，机器翻译，图像描述

LSTM(Long Short Term Memory)

In [16]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist=input_data.read_data_sets('MNIST_data',one_hot=True)
mnist

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x121917eb8>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x1219d4898>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x1219d4a58>)

In [21]:
#输入图片规格28x28
#输入一行，一行有28个数据
n_inputs=28
#一共28行
max_time=28

#隐藏层单元
lstm_size=100
#10个分类
n_classes=10

#每个批次50个样本
batch_size=50
#一共多少个批次
n_batch=mnist.train.num_examples//batch_size


x=tf.placeholder(tf.float32,[None,784])
y=tf.placeholder(tf.float32,[None,10])

weights=tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
biases=tf.Variable(tf.constant(0.1,shape=[n_classes]))

In [22]:
#RNN网络

def RNN(X,weights,biases):
    inputs=tf.reshape(X,[-1,max_time,n_inputs])
    #定义LSTM基本cell
    lstm_cell=tf.nn.rnn_cell.BasicLSTMCell(lstm_size)
    
    #final_state[0]:cell state
    #final_state[1]:hidden_state
    outputs,final_state=tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)

    results=tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
    return results


prediction=RNN(x,weights,biases)
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y,logits=prediction))
train_step=tf.train.AdamOptimizer(1e-4).minimize(loss)
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(prediction,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(5):
        for batch in range(n_batch):
            batch_xs,batch_ys=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys})
        
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
        print('Tter %s,Test accuracy:%s' % (epoch,acc))



ValueError: Variable rnn/basic_lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-11-40b64e1814aa>", line 10, in RNN
    outputs,final_state=tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
  File "<ipython-input-11-40b64e1814aa>", line 16, in <module>
    prediction=RNN(x,weights,biases)
  File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


# 又出现jupyter出现错误，然后在pyCharm可以正常运行的情况